# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,18.62,49,88,2.24,AU,1735709012
1,1,bilibino,68.0546,166.4372,-14.81,91,100,1.47,RU,1735709013
2,2,arapgir,39.0412,38.4952,-3.35,83,13,4.30,TR,1735709014
3,3,anadyr,64.7500,177.4833,-3.72,80,40,5.00,RU,1735709015
4,4,ushuaia,-54.8000,-68.3000,9.81,93,100,4.12,AR,1735709016


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_df.hvplot.points(
    "Lng", "Lat",  # Use longitude and latitude for plotting points
    geo=True,  # Enable geographic mapping
    size="Humidity",  # Point size represents humidity
    color="City",  # Color by city name for distinction
    tiles="OSM"  # Use OpenStreetMap as the base map
)

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) &  # Max temp greater than 21°C
    (city_data_df["Max Temp"] < 27) &  # Max temp less than 27°C
    (city_data_df["Wind Speed"] < 4.5) &  # Wind speed less than 4.5 m/s
    (city_data_df["Cloudiness"] == 0)  # No cloudiness
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()  # Remove rows with missing data

# Display sample data
ideal_cities_df.head()  # Show

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,terra rica,-22.7094,-52.6169,22.02,83,0,2.84,BR,1735709045
52,52,tura,25.5198,90.2201,23.05,40,0,1.42,IN,1735709075
53,53,kathor,21.3000,72.9333,25.89,46,0,2.73,IN,1735709076
150,150,monywa,22.1167,95.1333,25.50,45,0,2.94,MM,1735709192
213,213,sanibel,26.4490,-82.0223,22.03,97,0,2.96,US,1735709267


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()  # Select relevant columns

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""  # Create an empty column for hotel names

# Display sample data
hotel_df.head()  # Preview the new DataFrame

,City,Country,Lat,Lng,Humidity,Hotel Name
27,terra rica,BR,-22.7094,-52.6169,83,
52,tura,IN,25.5198,90.2201,40,
53,kathor,IN,21.3000,72.9333,46,
150,monywa,MM,22.1167,95.1333,45,
213,sanibel,US,26.4490,-82.0223,97,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000  # Search within 10,000 meters (10 km)
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "limit": 1,  # Limit results to the first hotel
    "apiKey": geoapify_key  # Use Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"  # Define search area (circle)
    params["bias"] = f"proximity:{lng},{lat}"  # Set bias toward the city center

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()  # Preview the updated DataFrame

Starting hotel search
terra rica - nearest hotel: No hotel found
tura - nearest hotel: Hotel Polo Orchid
kathor - nearest hotel: girnar farm delivery kamrej
monywa - nearest hotel: Hotel Chindwin
sanibel - nearest hotel: Sanibel Moorings
port elizabeth - nearest hotel: Waterford Hotel
ferkessedougou - nearest hotel: Ferké Hôtel Iroko
longboat key - nearest hotel: Zota Beach Resort
mathba - nearest hotel: No hotel found
batie - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
upington - nearest hotel: Die Eiland Holiday Ressort
pisco - nearest hotel: La Portada
east london - nearest hotel: No hotel found
al hazm - nearest hotel: No hotel found
sarupathar - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
27,terra rica,BR,-22.7094,-52.6169,83,No hotel found
52,tura,IN,25.5198,90.2201,40,Hotel Polo Orchid
53,kathor,IN,21.3000,72.9333,46,girnar farm delivery kamrej
150,monywa,MM,22.1167,95.1333,45,Hotel Chindwin
213,sanibel,US,26.4490,-82.0223,97,Sanibel Moorings


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_df.hvplot.points(
    "Lng", "Lat",  # Use longitude and latitude for plotting points
    geo=True,  # Enable geographic mapping
    size="Humidity",  # Point size represents humidity
    color="Country",  # Color by country for clarity
    tiles="OSM",  # Use OpenStreetMap as the base map
    hover_cols=["Hotel Name", "City"]  # Add hotel name and city to hover messages
)

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Hotel Name,City)